In [3]:
# !pip install tensorflow==2.15.0

In [4]:
# !pip install tensorboard

In [5]:
import pickle
import datetime
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

In [6]:
data = pd.read_csv('dataset/Churn_Modelling.csv')
print(data.shape)
data.head()

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### 1. Preprocessing

In [7]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.shape

(10000, 11)

In [8]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [9]:
data['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

In [10]:
# One hot geography, because through label encoder Germany as 2 will be considered of higher value than others
data['Geography'].nunique()

3

In [11]:
onehot_geography = OneHotEncoder(sparse_output=False) # drop='first'
geo_encode = onehot_geography.fit_transform(data[['Geography']])
geo_encode, onehot_geography.get_feature_names_out(['Geography'])

(array([[1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        ...,
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.]]),
 array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
       dtype=object))

In [12]:
df_geo = pd.DataFrame(geo_encode, columns=onehot_geography.get_feature_names_out(['Geography']))
df_geo.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [13]:
data = pd.concat([data.drop('Geography', axis=1), df_geo], axis=1)
data.shape

(10000, 13)

In [14]:
data.dtypes

CreditScore            int64
Gender                 int32
Age                    int64
Tenure                 int64
Balance              float64
NumOfProducts          int64
HasCrCard              int64
IsActiveMember         int64
EstimatedSalary      float64
Exited                 int64
Geography_France     float64
Geography_Germany    float64
Geography_Spain      float64
dtype: object

In [15]:
# ## Saving encoders
# with open('label_encoder_gender.pkl', 'wb') as file:
#     pickle.dump(label_encoder_gender, file)

# with open('onehot_encoder_geography.pkl', 'wb') as file:
#     pickle.dump(onehot_geography, file)

### 2. Modelling

In [16]:
X, y = data.drop('Exited', axis=1), data['Exited']
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=100, test_size=0.25)

# Scale these features
scaler = StandardScaler()
Xtrain = scaler.fit_transform(Xtrain)
Xtest = scaler.transform(Xtest)

In [17]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [19]:
# Model
model = Sequential([
    Dense(64, activation='relu', input_shape=(Xtrain.shape[1], )), # HL1 (Connected with input layer)
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid') # Output layer
])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
12*64+64, 32*64+32, 32+1

(832, 2080, 33)

In [22]:
# opt = tf.optimizers.Adam(learning_rate=0.01)
# loss = tf.keras.losses.BinaryCrossentropy()

In [23]:
# model.compile(optimizer=opt', loss=loss', metrics=['accuracy'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
# Logging
logs_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=logs_dir, histogram_freq=1)

In [25]:
## Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True) 
# patience - wait for atleast 5 iterations even after getting best weights

In [26]:
history = model.fit(
    Xtrain, ytrain, validation_data=(Xtest, ytest), epochs=100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100


235/235 [==============================] - 2s 5ms/step - loss: 0.4596 - accuracy: 0.7973 - val_loss: 0.4121 - val_accuracy: 0.8276
Epoch 2/100
235/235 [==============================] - 1s 3ms/step - loss: 0.3889 - accuracy: 0.8353 - val_loss: 0.3758 - val_accuracy: 0.8444
Epoch 3/100
235/235 [==============================] - 1s 4ms/step - loss: 0.3560 - accuracy: 0.8532 - val_loss: 0.3583 - val_accuracy: 0.8520
Epoch 4/100
235/235 [==============================] - 1s 3ms/step - loss: 0.3425 - accuracy: 0.8569 - val_loss: 0.3523 - val_accuracy: 0.8540
Epoch 5/100
235/235 [==============================] - 1s 3ms/step - loss: 0.3371 - accuracy: 0.8617 - val_loss: 0.3515 - val_accuracy: 0.8588
Epoch 6/100
235/235 [==============================] - 1s 3ms/step - loss: 0.3322 - accuracy: 0.8627 - val_loss: 0.3522 - val_accuracy: 0.8564
Epoch 7/100
235/235 [==============================] - 1s 3ms/step - loss: 0.3290 - accuracy: 0.8672 - val_loss: 0.3508 - val_accuracy: 0.85

In [27]:
model.save('model.h5')

d:\Programs\Anaconda\envs\gen_env\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [62]:
# Loading Tensorboard Extension
# %load_ext tensorboard
%reload_ext tensorboard

In [63]:
%tensorboard --logdir D:/Github/ML-AI-Projects/DL/Project_02/logs/fit/
# logs/fit/

Reusing TensorBoard on port 6006 (pid 4488), started 0:17:55 ago. (Use '!kill 4488' to kill it.)

### 3. Prediction Service

In [70]:
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gen = pickle.load(file)

with open('onehot_encoder_geography.pkl', 'rb') as file:
    onehot_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    standscaler = pickle.load(file)

pred_model = load_model('model.h5')

In [71]:
# Unseen Data Eg
input_data = {
    'CreditScore': 600, 'Geography': 'France', 'Gender': 'Male', 'Age': 40, 'Tenure': 3,
    'Balance': 60000, 'NumOfProducts': 2, 'HasCrCard': 1, 'IsActiveMember': 1, 'EstimatedSalary': 50000
}

df_pred = pd.DataFrame([input_data.values()], columns=input_data.keys())
df_pred.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [72]:
geo_code = onehot_geo.transform(df_pred[['Geography']])
df_geo = pd.DataFrame(geo_code, columns=onehot_geo.get_feature_names_out(['Geography']))
df_geo.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [73]:
df_pred = pd.concat([df_pred.drop('Geography', axis=1), df_geo], axis=1)
df_pred.shape

(1, 12)

In [74]:
df_pred['Gender'] = label_encoder_gen.transform(df_pred['Gender'])
df_pred.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [75]:
df_pred = standscaler.transform(df_pred)

In [76]:
prediction = model.predict(df_pred)
prediction

1/1 [==============================] - 0s 27ms/step


array([[0.03325391]], dtype=float32)

In [77]:
prediction[0][0]

0.03325391